In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131323 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p godrive
!google-drive-ocamlfuse godrive
print("Files in Drive:")
!ls godrive/

Files in Drive:
'07111540007002 (aea61266).pdf'
 07111540007002.pdf
'BENDAHARA PKMM 2015.FIX1.rar'
 CNN_Audio.ipynb
 data_train
'FINAL PROJECT MV 2018'
'Formulir tanpa judul (Tanggapan).ods'
'Formulir tanpa judul.zip'
'Foto Google'
'FOTO INDIVIDU ELEKTRO 2015'
 gpio.php
'How to get started with Drive'
'KRSBI BERODA ITS 2018'
'LAB SET 2018'
 labset.php
 labset.py
 main.c
'Pak Muhtadin'
 raspi.txt
'Salinan 22702 Muhammad Azhar Ismail.JPG'
'[SIAKAD-ITS] FRS Versi Cetak.pdf'
'TRANSKRIP MATA KULIAH.pdf'


In [0]:
import librosa
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt

DATA_PATH = '/content/godrive/data_train/data_train/'

def wav2mfcc(file_path, max_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=44100, n_mfcc=20, hop_length= 2048)
    # print(mfcc.shape)
    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]

    return mfcc

def get_labels(path=DATA_PATH):

    labels = os.listdir(path)
    return labels


def save_data_to_array(path=DATA_PATH, max_len=11):
    labels = get_labels(path)

    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        #for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
        for wavfile in wavfiles:
            mfcc = wav2mfcc(wavfile, max_len=max_len)
            mfcc_vectors.append(mfcc)
        np.save(label + '.npy', mfcc_vectors)

def get_train_test(split_ratio=0.8, random_state=25):
    labels = get_labels(DATA_PATH)
    label_all = []
    for label in labels:
        data_label = label.split("_")
        label_all.append(data_label)
    mlb = MultiLabelBinarizer()
    mlb.fit(label_all)
    label_data = mlb.classes_
    print(label_data)
    # Getting first arrays
    X = np.load(labels[0] + '.npy')
    y = mlb.transform([labels[0].split("_")])
    for i in range(len(X)-1):
        y = np.vstack((y, mlb.transform([labels[0].split("_")])))


    # Append all of the dataset into one single array, same goes for y
    for label in labels[1:]:
        x = np.load(label + '.npy')
        X = np.vstack((X, x))
        for i in range(len(x)):
            y = np.vstack((y, mlb.transform([label.split("_")])))

    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size=(1 - split_ratio), random_state=random_state, shuffle=True)

save_data_to_array(path=DATA_PATH, max_len=25)
X_train, X_test, y_train, y_test = get_train_test()

X_train = X_train.reshape(X_train.shape[0], 20, 25, 1)
X_test = X_test.reshape(X_test.shape[0], 20, 25, 1)

model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(20, 25, 1)))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.25))
# model.add(Conv2D(32, kernel_size=(2, 2), activation='sigmoid'))
# model.add(MaxPooling2D(pool_size=(1, 1)))
# model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='sigmoid'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

print(model.summary())

history = model.fit(X_train, y_train, batch_size=None, steps_per_epoch=50, epochs=500,
                    verbose=1, validation_data=(X_test, y_test), validation_steps=500)

model.save('/content/godrive/data_train/data_train/Save_Model/model_multilabel_new.h5')

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

['Arif' 'Azhar' 'Diam' 'Kalibrasi' 'Kanan' 'Kiri' 'Maju' 'Mundur' 'Nisar'
 'Tendang']
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 19, 24, 32)        160       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 6, 32)          0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 4, 6, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 768)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               393728    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
________________________________________________________